**IMPORTANT** <br> <ul> <li> Do **NOT** replace or remove this notebook (ipynb file)! Each cell has unique nbgrader's metadata and ID which, if changed outside the nbgrader, cannot pass the tests. Do **NOT** change the name of the file!</li> <li> To receive any credit, don't forget to **SUBMIT** your notebook when you are done! You can have multiple submissions before the deadline; only the last one is saved, including its timestamp.</li> <li>Before submitting, **Validate** your notebook to check if your codes pass all visible tests. </li> <li>Make sure you fill in any cell with the comment `# your code here`. Remove or comment the command `fail()` (in R), or `raise NotImplementedError` (in Python) and place your code there </li> </ul>

In [ ]:
NAME = "Christian Olivares-Rodriguez"

---

# PART 1: Naive Bayes in Python


- We will implement Naive Bayes in Python using built-in Scikit-Learn methods in the first part of this homework.

<br>

Let's start by importing libraries. Run the following cell.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OrdinalEncoder # for encoding categorical features from strings to number arrays
from sklearn.naive_bayes import CategoricalNB

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sklearn

## Read in and quick look at the data 

<br>

We use the dataset `Heart.csv`, from *An Introduction to Statistical Learning with Applications in R* (ISLR): <https://www.statlearning.com>. <br>

Run the following cell to explore the data set.

In [2]:
df = pd.read_csv('Heart.csv', sep=',')

print(df.shape)
df.head()


(303, 15)


,PatientID,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


Each row (i.e. observation) represents a single patient. The last variable, `AHD`, is binary (Yes/No) and indicates whether there is angiographic evidence of heart disease. This variable is our output categorical variable. All other variables except the first one (PatientID) are our predictors/features.<br>

<br>

### Goal

Our goal is to use the Naive Bayes method to classify patients by the `AHD` label. That is, based on the predictor variables (features), we want to predict whether the patient has angiographic heart disease (`AHD = "Yes"`) or not (`AHD = "No"`).

<br>

## Data munging and cleaning

<br>

The variable `PatientID` is not a predictor, so we remove it from our data frame `df`.

We can use, for example, the `pandas` method `drop()` to drop

`df = df.drop(columns = ??)`,

or `iloc()` to extract needed data:

`df = df.iloc[??, ??]`

Simply run the following to drop:

In [3]:
df = df.drop(columns=["PatientID"])

print(df.shape)
df.head()

(303, 14)


,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


## Train/Test Split

As usual, we should perform a train/test split. The following cell does just this. We could, and usually do, simply use `train_test_split` from `sklearn.model_selection`, but let's mix it up a bit just as a reminder of the process.

As a result, we get `train_df` and `test_df`. These DataFrames contain both our features and output.

In [4]:
# This cell performs a train/test split
#####

# set seed
np.random.seed(123)

# Randomize the DataFrame (i.e. shuffle the rows); 
df_randomized = df.sample(frac=1)  #could also set seed by argument random_state=123

# Calculate index for split (size of train data) -take first 70% of the data for training set
N = round(len(df_randomized) * 0.7)

# Split into training and test dataframes
# We can throw in reseting the index as well; drop=True means don't keep the old index as a column
train_df = df_randomized[:N].reset_index(drop=True)

test_df = df_randomized[N:].reset_index(drop=True)

# Check the shapes of our new dataframes
print(train_df.shape)
print(test_df.shape)

# Can confirm that setting the random number seed makes results reproducible by running this cell a few times:
train_df.head()

(212, 14)
(91, 14)


,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,56,0,nontypical,140,294,0,2,153,0,1.3,2,0.0,normal,No
1,44,1,asymptomatic,120,169,0,0,144,1,2.8,3,0.0,fixed,Yes
2,42,1,nonanginal,130,180,0,0,150,0,0.0,1,0.0,normal,No
3,40,1,asymptomatic,152,223,0,0,181,0,0.0,1,0.0,reversable,Yes
4,63,0,nonanginal,135,252,0,2,172,0,0.0,1,0.0,normal,No


## Imputation

We have a handful of missing values in our dataframe. Recall our three basic options for dealing with missing data:

1. Drop the missing observations
   - `drop()`
2. Drop the whole attribute
   - `dropna()`
3. Impute the missing values to something else. Common choices = 0, mean/median, most common value, or some clustering or regression approach
   - `fillna()`
   - Other imputer

Let's see where we have nulls in our training data:

In [5]:
train_df.isnull().sum(axis=0) #sum along first dimension/axis, so, indexed by 0 (axis=0)

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           3
Thal         1
AHD          0
dtype: int64

### Imputation for Continuous

Looks like we are missing a few values for both a continuous (`Ca`) and categorical (`Thal`) value.  The following cells impute our numeric data, using the `median`.

Now, it actually turns out that `Ca` is really categorical, but we'll get the same result whether we use the median (0) or most frequent value (0), so for the sake of practice we will treat it as continuous in the following.

In [6]:
#Let's fill with SimpleImputer:
from sklearn.impute import SimpleImputer

imputer_continuous = SimpleImputer(strategy='median')

#Restrict to only the numeric data types:
train_numeric = train_df.select_dtypes(include=[np.number])

#Can fit transform on the training data:
impute_results = imputer_continuous.fit_transform(train_numeric)

#We get a numpy array as output

#Convert to dataframe:
train_numeric = pd.DataFrame(impute_results, columns = train_numeric.columns,
                              index = train_numeric.index)

#We got?
train_numeric.head()

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca
0,56.0,0.0,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0
1,44.0,1.0,120.0,169.0,0.0,0.0,144.0,1.0,2.8,3.0,0.0
2,42.0,1.0,130.0,180.0,0.0,0.0,150.0,0.0,0.0,1.0,0.0
3,40.0,1.0,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0
4,63.0,0.0,135.0,252.0,0.0,2.0,172.0,0.0,0.0,1.0,0.0


In [7]:
#Check that it worked?
train_numeric.isnull().sum(axis=0)

Age        0
Sex        0
RestBP     0
Chol       0
Fbs        0
RestECG    0
MaxHR      0
ExAng      0
Oldpeak    0
Slope      0
Ca         0
dtype: int64

### <font style="color: red">Question 1.</font>


## Now, do it for the categorical data

Following the steps for numeric data, create a DataFrame named `train_categorical` that consists of the three categorical columns of type `object`, i.e. `ChestPain`, `Thal`, and `AHD`. Use `SimpleImputer` to impute any missing values, using the `most_frequent` strategy.

Your end result should be a **DataFrame** named `train_categorical` that has no null values.

In [40]:
# your code here

train_categorical = train_df[['ChestPain', 'Thal', 'AHD']]

imputer = SimpleImputer(strategy='most_frequent')

train_categorical_imputed = imputer.fit_transform(train_categorical)

train_categorical = pd.DataFrame(train_categorical_imputed, columns=['ChestPain', 'Thal', 'AHD'])

In [41]:
## test whether train_categorical is pandas DataFrame
assert isinstance(train_categorical, pd.core.frame.DataFrame)

## test whether train_categorical has the correct shape
assert train_categorical.shape == (212, 3)

## test whether train_categorical has any NaNs
assert train_categorical.isna().sum().sum() == 0

## test whether there are 114 instances of "normal" in column "Thal"
assert (train_categorical.Thal == 'normal').sum() == 114

### Get everything back together

Let's put everything back in the new DataFrame `train_df_filled`:

In [42]:
#Put back into our train_df, which we'll now call train_df_filled:
#######
train_df_filled = train_df.copy()

train_df_filled[train_numeric.columns] = train_numeric[train_numeric.columns]
train_df_filled[train_categorical.columns] = train_categorical[train_categorical.columns]

train_df_filled.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,56.0,0.0,nontypical,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,normal,No
1,44.0,1.0,asymptomatic,120.0,169.0,0.0,0.0,144.0,1.0,2.8,3.0,0.0,fixed,Yes
2,42.0,1.0,nonanginal,130.0,180.0,0.0,0.0,150.0,0.0,0.0,1.0,0.0,normal,No
3,40.0,1.0,asymptomatic,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0,reversable,Yes
4,63.0,0.0,nonanginal,135.0,252.0,0.0,2.0,172.0,0.0,0.0,1.0,0.0,normal,No


Confirm that there are no NaNs in the DataFrame (note if the sum is not 0, you must have done something wrong in creating `train_categorical`):

In [43]:
#This should be 0 if all went right
train_df_filled.isna().sum().sum()

0

### Note: Check the proportions of our labels:

Let's see the proportions of labels (Yes/No) in `AHD`.

In [44]:
## Relative frequency of No and Yes in the training data
## Data is fairly balanced
train_df_filled['AHD'].value_counts(normalize=True)

No     0.54717
Yes    0.45283
Name: AHD, dtype: float64

## Fit Naive Bayes Using the training data


Now, let's construct a model and fit it on the training data.

## Create the Naive Bayes model

<br>

First, we create pandas **DataFrame** `X_train` and  pandas **Series** `y_train`. The DataFrame `X_train` should consist of only features/predictors of the training data, not the output `AHD`, while `y_train` should consist of the corresponding labels (i.e. column `AHD`) of the training data.

After creation, we print out the first couple of observations for both objects to check that everything seems okay (sanity check).

In [47]:
#Our training X and training y
####

X_train = train_df_filled.iloc[:,:-1]
y_train = train_df_filled['AHD']


display(X_train.head())

display(y_train.head())

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal
0,56.0,0.0,nontypical,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,normal
1,44.0,1.0,asymptomatic,120.0,169.0,0.0,0.0,144.0,1.0,2.8,3.0,0.0,fixed
2,42.0,1.0,nonanginal,130.0,180.0,0.0,0.0,150.0,0.0,0.0,1.0,0.0,normal
3,40.0,1.0,asymptomatic,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0,reversable
4,63.0,0.0,nonanginal,135.0,252.0,0.0,2.0,172.0,0.0,0.0,1.0,0.0,normal


0     No
1    Yes
2     No
3    Yes
4     No
Name: AHD, dtype: object

### <font style="color: red">Question 2.</font>

Now, create pandas DataFrame `X_test` and pandas Series `y_test`. The DataFrame `X_test` should consist of all the features/predictors values from the test data (without output `AHD`). The `y_test` Series should consist of just the output values (i.e. labels) from the test data.

**Remember you need to first impute missing values, *using the imputers fit to the training data*.**

In [95]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
numeric_columns = X_train.select_dtypes(include=[np.number]).columns
X_train_numeric = X_train[numeric_columns]
X_test_numeric = X_test[numeric_columns]
# Initialize and apply the StandardScaler to the numeric data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_numeric)
X_test = scaler.transform(X_test_numeric)

if not isinstance(X_test, pd.core.frame.DataFrame):
    X_test = pd.DataFrame(X_test_scaled, columns=numeric_columns, index=X_test.index)

In [96]:
## test whether X_test is pandas DataFrame
assert isinstance(X_test, pd.core.frame.DataFrame)

## test whether the first 4 values of column Age are correct
assert sum(X_test['Age'].iloc[:4] != [47, 55, 66, 42]) == 0

## test whether the first 4 values of column ChestPain are correct
assert sum(X_test['ChestPain'].iloc[:4] != ['nonanginal','asymptomatic','typical','typical']) == 0

## test whether the first 4 values of column Chol are correct
assert np.linalg.norm(np.array(X_test['Chol'].iloc[:4]) - np.array([243, 353, 226, 244]), ord=2) < 1.e-5

AssertionError: 

In [60]:
## test whether y_test is a pandas Series
assert isinstance(y_test, pd.core.series.Series)


## test that first 4 and last 4 values of y_test are correct
assert sum(y_test.iloc[:4] != ['Yes', 'Yes', 'No', 'No']) == 0
assert sum(y_test.iloc[-4:] != ['Yes', 'No', 'No','No']) == 0

AssertionError: 

### Encoding and training

To prepare for training, we encode our training labels (`y_train`) to either a numeric 0 or 1 (Bernoulli random variable, or binary encoding), and also use a numpy array instead of a pandas Series. The 0-1 coded values are stored in the variable `y_train_int`.

One way to do this is with a little list comprehension

`y_train_int = np.array([int(train_y[i]=='Yes') for i in range(len(train_y))])`

Or like this:

`y_train_int = (y_train == 'Yes').astype(int).values`

However, we will demonstrate the `LabelEncoder()` function from `sklearn.preprocessing` submodule.

We'll go ahead and also create `y_test_int` in the following cell.

In [ ]:
from sklearn.preprocessing import LabelEncoder

#0-1 encoding train labels
y_train_int = LabelEncoder().fit_transform(y_train)

#Do for the testing set too:
y_test_int = LabelEncoder().fit_transform(y_test)

y_train_int[:5] #print first 5 of y_train_int (check that No=0, Yes=1)


### Naive Bayes with mixed variables

In our dataset `Heart` we have both discrete and continuous features. Submodule `sklearn.naive_bayes` does not handle this situation very well automatically. There are several ways to proceed, each simplifying the situation. One way is to bin/bucketize the numerical features. That is, for each numerical feature, we group its values into bins, and each bin is represented by a single number from 0, 1, ... , n, where n is the number of bins (i.e. categories) for this feature.

To do this, then, we need to first bucketize each numeric feature. We'll use a custom transformer for this.

Then we use the `OrdinalEncoder` function from the submodule `sklearn.preprocessing` for categorical variables that are objects.

The following series of cells creates `X_train_prepared`, which has binned versions of the continuous variables, and encoded versions of the categorical variables.

In [ ]:
##Let's make a custom transformer for binning/bucketizing
#########

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array

class QuantileBucketer(BaseEstimator, TransformerMixin):
    def __init__(self, num_quantiles=10, random_state=42):
        self.num_quantiles = num_quantiles
        self.random_state = random_state

    def fit(self, X, y=None):
        #Extract feature names from columns:
        if (isinstance(X, pd.DataFrame)):
            self.feature_names = np.array(X.columns)
        #If no columns, just 0 through n
        else:
            self.feature_names = np.array(range(0, X.shape[1])).astype(str)

        
        #Bucketize the data
        
        X = check_array(X)
        
        #Go through each column:
        #Will have list of lists for the bins:
        self.bins_ = []
        
        #Do a quantile cut
        for i in range(np.shape(X)[1]):
            X_bucket, bins = pd.qcut(X[:,i],
                                 q=self.num_quantiles, labels=False, retbins=True, duplicates='drop')

            #Save the bins
            #We need to make sure all the testing data will go into a bin, so set the left and right limits to -/+ infinitiy
            bins[0] = -np.inf
            bins[len(bins)-1] = np.inf
            
            self.bins_.append(bins)
        
        return self #Always return self

    
    def transform(self, X):

        X = check_array(X)
        
        #Cut the variables based on the buckets trained in the fit
        for i in range(np.shape(X)[1]):
            X[:, i] = pd.cut(X[:,i], bins=self.bins_[i], labels=False)
            
        #Return bucketized X:
        return X


    def get_feature_names_out(self, names=None):
        #Just return the feature names used to create:
        return self.feature_names
            

In [ ]:
##Our continuous columns for binning:
##Note that some of the numeric are actually categorical already:
##We'll bucket into 10 quantiles, if possible
bucket = QuantileBucketer(num_quantiles=10)

bucket_cols = ['Age', 'RestBP', 'Chol', 'MaxHR', 'Oldpeak']

#We're going to cheat a little and train on all the data so that the whole range is included:
#bucket.fit(pd.concat((X_train[bucket_cols], X_test[bucket_cols])))
bucket.fit(X_train[bucket_cols])


#Make X_train_prepared as follows:
X_train_prepared = X_train.copy()
X_train_prepared[bucket_cols] = bucket.transform(X_train[bucket_cols])

#What we got?
X_train_prepared.head()

In [ ]:
#Create an OrdinalEncoder for ChestPain and Thal:
#######

from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

#And encode!
X_train_prepared[['ChestPain', 'Thal']] = ord_enc.fit_transform(X_train_prepared[['ChestPain', 'Thal']])

In [ ]:
##Result?

X_train_prepared.head()

### <font style="color: red">Question 3.</font>


## Create `X_test_prepared`

Use the bucket transformer and ordinal encoder created in the above cells to create `X_test_prepared`. You should use the same binning scheme and ordinal encoding as for the training data.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
## test whether X_test_prepared is pandas DataFrame
assert isinstance(X_test_prepared, pd.core.frame.DataFrame)

## test whether the first 4 values of column Age are correct
assert sum(X_test_prepared['Age'].iloc[:4] != [2, 4, 8, 0]) == 0

## test whether the first 4 values of column ChestPain are correct
assert sum(X_test_prepared['ChestPain'].iloc[:4] != [1, 0, 3, 3]) == 0

## test whether the first 4 values of column Chol are correct
assert np.linalg.norm(np.array(X_test_prepared['Chol'].iloc[:4]) - np.array([5, 9, 3, 5]), ord=2) < 1.e-5

## Creating and fitting the model

We use the `CategoricalNB()` function from `sklearn.naive_bayes` submodule, which requires encoding, which is why we did the previous step.  We can create and train as follows:

In [ ]:
model = CategoricalNB()  #create model object
model.fit(X_train_prepared, y_train_int) # fit on train data


### Predicting

To measure performance, we should use test data. That will be your job in a moment. However, here, we make predictions for each training observation. Again, this is not as reliable estimate of true performance as when test data are used. The main purpose of using training rather than test data is to ensure the model did reasonably on in-sample data, and as a model for making prediction on test data (out-of-sample).

**Note**: As the model has been specifically constructed to minimize error with respect to the training data, we are vulnerable to overfitting (recall the **bias-variance tradeoff** phenomenon). As the notion of bias-variance tradeoff suggests, at some point a better fit to training data implies worse performance on other possible datasets.

In [ ]:
yhat_train = model.predict(X_train_prepared) # predict on train data

#### Confusion Matrix

To assess performance, we can take a look at confusion matrix for the training data:

In [ ]:
## confusion matrix using pandas method crosstab
pd.crosstab(y_train, yhat_train)


In [ ]:
#Alternative way, just using scikit-learn's confusion_matrix function:
confusion_matrix(y_train_int, yhat_train)


Note that the accuracy score, i.e. the proportion of correctly predicted outputs is ${103+78 \over103+13+18+78} \approx 85.38\%$. 

**Caveat** Due to bias-variance tradeoff, we know that this reasonably high score does not necessarily guarantee a good prediction on new (out-of-sample) data. It might be just due to overfitting - i.e. if the model is too flexible, it will easily fit the train data very well. Again, good performance on out-of-sample data are not guaranteed. This is the reason why we want to test performance on new data (test data), that were not used for fitting the model.

The accuracy score can also be computed using `accuracy_score()` function from the `sklearn.matrics` module.

In [ ]:
accuracy_score(y_train_int, yhat_train)

### <font style="color: red">Question 4.</font>


Using the `X_test_prepared` DataFrame and `y_test_int` array from above, create predictions for the testing dataset.

Create array `yhat_test`, using the model trained on training data, and the processed feature values for the test data.


In [ ]:

# your code here
raise NotImplementedError


In [ ]:
## test whether first 10 values of column yhat_test are correct
assert sum(yhat_test[:10] != [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]) == 0


### <font style="color: red">Question 5.</font>

<br>

Compute the confusion matrix (for test data) and store it as a variable `cm`.

**Hint**: the entry in the 1st row and 1st column (predicted value=No, actual AHD value=No) should be 45.

In [ ]:

# your code here
raise NotImplementedError

In [ ]:
## check whether the determinant of the confusion matrix is 1503

assert abs(np.linalg.det(np.array(cm)) - 1503) < 1.e-5


In [ ]:
## check whether the answer is correct (hidden test)



### <font style="color: red">Question 6.</font>

#### Accuracy Score

Compute the accuracy score, i.e. the proportion of the correctly predicted outputs. Store this value as python variable `acc`. So, your answer should look like

`acc = <some expression>`

Do **NOT** round your answer.

In [ ]:

# your code here
raise NotImplementedError


In [ ]:
## test whether 4th and 5th decimal digits of acc are 13

assert np.mod(int(np.floor(acc * 10**5)), 100) == 13

In [ ]:
## test whether acc is correct (hidden tests)




**Remark**

If you got `acc` right, you will notice that the accuracy score using test data is actually slightly higher than for the training data. While it is possible (depending on randomization of training and test samples) that the accuracy score will happen to be higher when measured on test data than on train data, on average, it is expected to be lower on the test data.

### <font style="color: red">Question 7.</font>

### TPR and TNR (Sensitivity and Specificity)

Now, let us calculate the true positive rate (TPR, also known as the sensitivity or the recall), and the true negative rate (TNR, also known as specificity).  The former gives us the probability that, in this case, given angiographic heart disease is present, our model correctly gives us a positive.  The latter gives the probability that our model returns a negative result, given that angiographic heart disease is absent.

We can calculate these quantities as follows:

$$
\text{Sensitivity = TPR = } \frac{\text{TP}}{\text{TP + FN}}
$$

$$
\text{Specificity = TNR = } \frac{\text{TN}}{\text{TN + FP}}
$$

Do so, and assign your answers to the variables `TPR` and `TNR`.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
## test whether the 2nd and 3rd decimal digits of TPR are 90

assert np.mod(int(np.floor(TPR * 10**3)), 100) == 90

In [ ]:
## test whether the 2nd and 3rd decimal digits of TNR are 37

assert np.mod(int(np.floor(TNR * 10**3)), 100) == 37

In [ ]:
## test whether TPR and TNR are correct (hidden tests)


### <font style="color: red">Question 8.</font>

## Precision/Recall and the F1 Score

Calculate and report the precision (positive predictive value), recall (sensitivity), and F1 score (harmonic mean of precision and recall) for the model on testing data.

Assign your answers to the variables `precision`, `recall`, and `f1`.

In [ ]:

# your code here
raise NotImplementedError

In [ ]:
## test whether the 2nd and 3rd decimal digits of precision are 18

assert np.mod(int(np.floor(precision * 10**3)), 100) == 18

In [ ]:
## test whether the 2nd and 3rd decimal digits of recall are 90

assert np.mod(int(np.floor(recall * 10**3)), 100) == 90

In [ ]:
## test whether the 2nd and 3rd decimal digits of f1_score are 50

assert np.mod(int(np.floor(f1 * 10**3)), 100) == 50

In [ ]:
## test whether precision, reall, and f1_score are correct (hidden tests)


### <font style="color: red">Question 9.</font>

## ROC Curve

Construct ROC curves summarizing model performance on both the training and testing datasets. Your result should mimic the following:

<img src="ROC_Bayes.png" alt="ROC Curve" style="width:600px;"/>


In [ ]:

# your code here
raise NotImplementedError

### <font style="color: red">Question 10.</font>

## Precision-Recall Curve

Construct Precision-Recall curves summarizing model performance on both the training and testing datasets. Your result should mimic the following:

<img src="PR_Bayes.png" alt="PR Curve" style="width:600px;"/>


In [ ]:

# your code here
raise NotImplementedError

# PART 2: KNN Regression/Classification in Python + Hyperparameter Tuning


In this part of the homework, we'll train KNN regression and classification models in Python, as well as find the best value for $k$, the number of nearest neighbors, using functions available through scikit-learn.

Run the following cell to import some more libraries/objects that we'll need:

In [ ]:
#Some helper functions to split and scale our data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#KNN Classifier and Regression models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

#Allows us to exhaustively search for best hyperparameter using cross-validation
from sklearn.model_selection import GridSearchCV

## PART 2A. KNN Regression

Now, let's load the `Maricopa_AQ.csv` dataset: This gives mean daily values of the four so-called critera gases, O$_3$, NO$_2$, SO$_2$, and CO, in Maricopa county from 2000 to 2016. Filtered down from original dataset available at https://www.kaggle.com/datasets/sogun3/uspollution, itself scraped from the EPA Air Quality System (https://www.epa.gov/aqs).

Our output variable will be `NO2 Mean`, while our predictors will be, **in this exact order in our DataFrame,** `O3 Mean`, `SO2 Mean`, and `CO Mean`. Run the following cell to load the data.

In [ ]:
df = pd.read_csv('Maricopa_AQ.csv')
df.head()

### <font style="color: red">Question 1.</font>

### Train/Test Split

- Use Scikit-learn's `train_test_split` function to give a simple randomized split. Do the following:

1. Set the random state to $42$
2. Hold back 30% of the data for testing

**Create the following variables:**

1. `X_train`
2. `X_test`
3. `y_train`
4. `y_test`

Your inputs (features, predictors) should be, **in this exact order in the DataFrame**

`O3 Mean`, `SO2 Mean`, and `CO Mean`

Your output (target) should be:

`NO2 Mean`

In [ ]:

# your code here
raise NotImplementedError


In [ ]:
## test whether the first 2 rows of X_train are correct:
c = np.array([[0.014333, 1.818182, 1.155556],
       [0.030625, 3.291667, 1.216667]])

assert np.isclose(X_train.iloc[:2,:].values, c).sum() == 6

## test whether the first 2 rows of y_train are correct:
assert np.isclose(y_train[:2].values, np.array([29.136364, 26.625])).sum() == 2


c = np.array([[0.031167, 1.25    , 0.420833],
       [0.014792, 3.416667, 1.4125  ]])

## test whether the first 2 rows of X_test are correct:
assert np.isclose(X_test.iloc[:2,:].values, c).sum() == 6

## test whether the first 2 rows of y_test are correct:
assert np.isclose(y_test[:2].values, np.array([15.75, 34.541667])).sum() == 2


### <font style="color: red">Question 2.</font>

### Rescale the Data

Note that the scales for our predictor values are quite different. Therefore, it is usually advisable to rescale data. We can rescale any feature to have mean 0 and unit variance with the operation

$$
z = \frac{x - \mu}{\sigma},
$$
where $\mu$ and $\sigma$ are the mean and standard deviation of the feature $x$, respectively.  We can either do this manually, or we can use `StandardScaler` from scikit-learn.

Use `StandardScaler()` to rescale your `X_train` and `X_test` variables. Note once again that you should *fit* the scaler to the `X_train` data (not the whole dataset, and not the test dataset), then use the fitted scaler to transform both the `X_train` and `X_test` variables.

Simply leave `X_train` and `X_test` and numpy arrays.

In [ ]:

# your code here
raise NotImplementedError

In [ ]:
## test whether the first 2 rows of (scaled) X_train are correct:
c = np.array([[-0.83998391,  0.10125774,  1.50787084],
       [ 0.60829962,  1.2631331 ,  1.66856412]])

assert (np.isclose(X_train[:2,:],c)).sum() == 6


c = np.array([[ 0.65648092, -0.34676627, -0.42410626],
       [-0.79918093,  1.36169836,  2.18351312]])

## test whether the first 2 rows of (scaled) X_test are correct:
assert (np.isclose(X_test[:2,:],c)).sum() == 6


### <font style="color: red">Question 3.</font>

### Use GridSearchCV to Find Optimal Hyperparameter $k$/`n_neighbors` for a KNN Regressor

We wish to train a `KNeighborsRegressor` on this data, but choose an optimal hyperparameter `n_neighbors`. Use the Scikit-learn `GridSearchCV()` object to evaluate a `KNeighborsRegressor()` model performance using k-folds cross-validation for $k$ (`n_neighbors`) between 1 and 100, inclusive, and with 10 folds.

Create a `GridSearchCV` object named `grid_search` and train it on your (scaled) training data.

<br>

Once you've fit `grid_search`, plot the mean score function ($R^2$ by default) across folds against the number of neighbors. You will need to use results found in `grid_search.cv_results_`.  Your plot should resemble:

<img src="k_curve.png" style="width: 450px;"/>

In [ ]:

# your code here
raise NotImplementedError


<br>

Now, extract the best `n_neighbors` parameter from `grid_search`, and assign this (integer) to the variable `k_best`.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
## test whether the k_best is between 30 and 40

assert ((30 <= k_best) and (40 >= k_best))


In [ ]:
## check whether k_best is correct (hidden test)


### <font style="color: red">Question 4.</font>

### Train Model, Predict

Create a `KNeighborsRegressor` model object named `knn_model` with `n_neighbors` set to the `k_best` you found above, and fit the model using your training dataset.

Once you've trained your model, make predictions for your test dataset. Store the predicted values in the variable `test_preds`. This should be a `numpy` array.


In [ ]:

# your code here
raise NotImplementedError

In [ ]:
## test whether the first 5 entries of test_preds are correct:

assert np.isclose(test_preds[:5], np.array([18.95303884, 37.14603256, 19.27508081,  9.98867597, 20.96954184])).sum() == 5


### <font style="color: red">Question 5.</font>

### Plot Performance

Plot predicted vs. actual output for your test data.  That is, to get a visual sense of how well the model performed, one can plot the predicted vs. true values. Perfect predictions should fall on straight line with slope 1.

Make such a plot (with appropriate labels, etc.), which should resemble the following:

<img src="y_vs_y.png" style="width:475px;"/>

In [ ]:
# your code here
raise NotImplementedError


## PART2B. KNN Classification

Now, let's train a KNN Classifer, using the `houseliving.csv` dataset, already introduced in class. Our output variable will be `house`, which is $1$ if the respondent owns their house, and $0$ else. Our predictors will be, **in this exact order in our DataFrame,** `salary` and `age`.

Run the cell below to import the data.


In [ ]:
df = pd.read_csv('houseliving.csv')

df.head()

In [ ]:
## Also note the balance of house values:
##

df['house'].value_counts(normalize=True)

### <font style="color: red">Question 6.</font>

### Train/Test Split Once Again

Once again, use Scikit-learn's `train_test_split` function to give a simple randomized split. Do the following:

1. Set the random state to $42$
2. Hold back 30% of the data for testing

**Create the following variables:**

1. `X_train`
2. `X_test`
3. `y_train`
4. `y_test`

Your inputs (features, predictors) should be, **in this exact order in the DataFrame**

`salary` and `age`

Your output (target) should be:

`housing`

In [ ]:
# your code here
raise NotImplementedError


In [ ]:
## test whether the first 2 rows of X_train are correct:
c = np.array([[3.3000e+04, 4.6000e+01],
       [4.8695e+04, 4.1000e+01]])

assert np.isclose(X_train.iloc[:2,:].values, c).sum() == 4

## test whether the first 10 rows of y_train are correct:
assert np.isclose(y_train[:10].values, np.array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0])).sum() == 10


c = np.array([[9.8305e+04, 3.9000e+01],
       [2.7000e+04, 4.3000e+01]])

## test whether the first 2 rows of X_test are correct:
assert np.isclose(X_test.iloc[:2,:].values, c).sum() == 4

## test whether the first 10 rows of y_testn are correct:
assert np.isclose(y_test[:10].values, np.array([0, 0, 1, 1, 0, 0, 1, 1, 0, 1])).sum() == 10


### <font style="color: red">Question 7.</font>

### Rescale the Data

Again, as before, use `StandardScaler` to rescale your `X_train` and `X_test` variables. Note that you should *fit* the scaler to the `X_train` data (not the whole dataset, and not the test dataset), then use the fitted scaler to transform both the `X_train` and `X_test` variables.

This time, **assign the scaled variables to `X_train_scaled` and `X_test_scaled`**. We want to preserve our original DataFrames for plotting below.

Leave `X_train` and `X_test` and numpy arrays.

In [ ]:

# your code here
raise NotImplementedError

In [ ]:
## test whether the first 2 rows of X_train_scaled are correct:
c = np.array([[-1.07742064, -0.04976943],
       [-0.65707682, -0.51053802]])

assert (np.isclose(X_train_scaled[:2,:],c)).sum() == 4


c = np.array([[ 0.67157924, -0.69484546],
       [-1.23811276, -0.32623059]])

## test whether the first 2 rows of X_test_scaled are correct:
assert (np.isclose(X_test_scaled[:2,:],c)).sum() == 4

### <font style="color: red">Question 8.</font>

### Use GridSearchCV to Find Optimal Hyperparameter

Use the scikit-learn `GridSearchCV` object to evaluate the model using k-folds cross-validation for $k$ (`n_neighbors`) between **1 and 50**, inclusive, and with **5 folds**.

This time, you must pass `KNeighborsClassifier()` to `GridSearchCV()` (NOT `KNeighborsRegressor`). Use your *scaled* training data to determine the optimal `n_neighbors` hyperparameter.

Assign the best value of `n_neighbors` to the variable `k_best`, as an **integer**.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
## test whether the k_best is between 40 and 50

assert ((40 <= k_best) and (50 >= k_best))

In [ ]:
## check whether k_best is correct (hidden test)


### <font style="color: red">Question 9.</font>

### Train Model, Predict

Create a `KNeighborsClassifier` model object named `knn_model` with `n_neighbors` set to `k_best` determined above, and fit the model using your scaled training dataset.

**Name your model `classifier`.**

Once you've trained your model, make predictions for your test dataset. Store the predicted values in the variable `test_preds`. This should be a `numpy` array.


In [ ]:
# your code here
raise NotImplementedError


In [ ]:
## test whether the model `classifier` exists:
assert (classifier)

## and that it is of the correct type:
assert type(classifier) == sklearn.neighbors._classification.KNeighborsClassifier

In [ ]:
## test whether the first 15 entries of test_preds are correct:

assert np.isclose(test_preds[:15], np.array([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])).sum() == 15


### <font style="color: red">Question 10.</font>

### Characterize Accuracy

Determine the following metrics, using the test dataset, and assign them to the indicated variables:

1. The accuracy: `acc`
2. Sensitivity (True Positive Rate): `TPR`
3. Specificity (True Negative Rate): `FPR`

You may find `accuracy_score` and `confusion_matrix` from `sklearn.metrics` useful.

In [ ]:
# your code here
raise NotImplementedError


In [ ]:
## test whether 4th and 5th decimal digits of acc are 77
assert np.mod(int(np.floor(acc * 10**5)), 100) == 77

## test whether 4th and 5th decimal digits of TPR are 36
assert np.mod(int(np.floor(TPR * 10**5)), 100) == 36

## test whether 4th and 5th decimal digits of TNR are 50
assert np.mod(int(np.floor(TNR * 10**5)), 100) == 50

In [ ]:
## test whether acc is correct (hidden tests)


In [ ]:
## test whether TPR is correct (hidden tests)


In [ ]:
## test whether TNR is correct (hidden tests)


### <font style="color: red">Question 10.</font>

### Plot Results

Create a contour plot to demonstrate the area that is predicted by the model to give `house = 0` vs `house = 1`, along with the testing data. Color testing data points by their true `house` value (not predicted). Your plot should resemble the following:

<img src="knn_classifier_surface.png" style="width: 475px;"/>

In [ ]:
# your code here
raise NotImplementedError